In [1]:
import tensorflow as tf  # Importing TensorFlow library for deep learning operations
import numpy as np  # Importing NumPy library for numerical operations
import os  # Importing os module for interacting with the operating system
import cv2  # Importing OpenCV library for image processing
from time import time as time  # Importing time function from time module to calculate time taken for training
gpus = tf.config.list_physical_devices('GPU')
for gpu in gpus:
    print(gpu.name)

/physical_device:GPU:0
/physical_device:GPU:1


In [2]:
#train_path = os.path.join(os.getcwd(), "Train/")  # Constructing the path for the training images directory
#test_path = os.path.join(os.getcwd(), "Test/")  # Constructing the path for the testing images directory
train_path = os.path.join(os.getcwd(), "C:/Users/vedan/Desktop/NUV/Tyre_Guardian/images/Train/")
test_path = os.path.join(os.getcwd(), "C:/Users/vedan/Desktop/NUV/Tyre_Guardian/images/Test/")

In [3]:
def load_folder(path):
    x_data = []  # List to store images
    y_data = []  # List to store labels
    for folder in os.listdir(path):  # Iterating through each folder in the specified directory
        label = 0 if folder == "defective" else 1  # Assigning label 1 if folder name is "defective", else 0
        folder_path = os.path.join(path, folder)  # Constructing the path for the current folder

        for img in os.listdir(folder_path):  # Iterating through each image in the folder
            image_path = os.path.join(folder_path, img)  # Constructing the path for the current image
            image = cv2.imread(image_path)  # Reading the image using OpenCV
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Converting image to RGB color space
            #image = cv2.resize(image, (256, 256))  # Resizing the image to (256, 256)
            x_data.append(image)  # Appending the image to the list of images
            y_data.append(label)  # Appending the label to the list of labels
    return np.array(x_data), np.array(y_data)  # Converting lists to NumPy arrays and returning

In [4]:
x_train, y_train = load_folder(train_path)  # Loading training images and labels
x_test, y_test = load_folder(test_path)  # Loading testing images and labels
print(x_train.shape, y_train.shape, x_test.shape, y_test.shape)  # Printing shapes of the loaded data

(2307, 256, 256, 3) (2307,) (577, 256, 256, 3) (577,)


In [5]:
np.save("x_train.npy", x_train)  # Saving training images as numpy array
np.save("y_train.npy", y_train)  # Saving training labels as numpy array
np.save("x_test.npy", x_test)  # Saving testing images as numpy array
np.save("y_test.npy", y_test)  # Saving testing labels as numpy array

In [6]:
def load_data():
    x_train = np.load("x_train.npy")  # Loading training images
    y_train = np.load("y_train.npy")  # Loading training labels
    x_test = np.load("x_test.npy")  # Loading testing images
    y_test = np.load("y_test.npy")  # Loading testing labels

    return x_train, y_train, x_test, y_test  # Returning the loaded data

In [7]:
x_train, y_train, x_test, y_test = load_data()  # Loading data
print(x_train.shape, y_train.shape, x_test.shape, y_test.shape)  # Printing shapes of the loaded data

(2307, 256, 256, 3) (2307,) (577, 256, 256, 3) (577,)


In [8]:
# preprocessing
y_train.reshape(y_train.shape[0], 1)  # Reshaping training labels
y_test.reshape(y_test.shape[0], 1)  # Reshaping testing labels
x_train = x_train.astype('float32') / 255.0  # Normalizing training images
x_test = x_test.astype('float32') / 255.0  # Normalizing testing images

In [9]:
# define model
'''model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(64, kernel_size=(3, 3), strides=(1, 1), padding='same', activation='relu', input_shape=(256, 256, 3)),  # Adding a 2D convolutional layer with 64 filters, kernel size 3x3, ReLU activation, and input shape (256, 256, 3)
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2)),  # Adding a max pooling layer with pool size 2x2 and stride 2x2
    tf.keras.layers.Dropout(0.25),  # Adding a dropout layer with dropout rate 0.25
    tf.keras.layers.Conv2D(128, kernel_size=(3, 3), strides=(2, 2), activation='relu'),  # Adding another 2D convolutional layer with 128 filters, kernel size 3x3, ReLU activation, and stride 2x2
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2)),  # Adding another max pooling layer with pool size 2x2 and stride 2x2
    tf.keras.layers.Dropout(0.25),  # Adding another dropout layer with dropout rate 0.25
    tf.keras.layers.Conv2D(64, kernel_size=(3, 3), strides=(2, 2), padding='same', activation='relu', input_shape=(256, 256, 3)),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2)),  # Adding a max pooling layer with pool size 2x2 and stride 2x2
    tf.keras.layers.Dropout(0.25),
    tf.keras.layers.Flatten(),  # Flattening the output from the convolutional layers
    tf.keras.layers.Dense(128, activation='relu'),  # Adding a fully connected dense layer with 128 units and ReLU activation
    tf.keras.layers.Dense(1, activation='sigmoid')  # Adding the output layer with 1 unit and sigmoid activation for binary classification
])
'''
#Relu= rectified Linear unit f(x)=max(0,x) introduces non linearity
#strides= step size of kernel (x,y)
#padding= add or same
#units= nodes
#dense= fully connected layer
with tf.device('GPU:0'):
    start=time()
    model = tf.keras.Sequential([
        tf.keras.layers.Conv2D(64, kernel_size=(3, 3), strides=(1, 1), padding='same', activation='relu', input_shape=(256, 256, 3)),  # Adding a 2D convolutional layer with 64 filters, kernel size 3x3, ReLU activation, and input shape (256, 256, 3)
        tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2)),  # Adding a max pooling layer with pool size 2x2 and stride 2x2
        tf.keras.layers.Dropout(0.25),  # Adding a dropout layer with dropout rate 0.25
        tf.keras.layers.Conv2D(128, kernel_size=(3, 3), strides=(2, 2), activation='relu'),  # Adding another 2D convolutional layer with 128 filters, kernel size 3x3, ReLU activation, and stride 2x2
        tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2)),  # Adding another max pooling layer with pool size 2x2 and stride 2x2
        tf.keras.layers.Dropout(0.25),  # Adding another dropout layer with dropout rate 0.25
        tf.keras.layers.Conv2D(64, kernel_size=(3, 3), strides=(2, 2), padding='same', activation='relu', input_shape=(256, 256, 3)),
        tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2)),  # Adding a max pooling layer with pool size 2x2 and stride 2x2
        tf.keras.layers.Dropout(0.25),
        tf.keras.layers.Flatten(),  # Flattening the output from the convolutional layers
        tf.keras.layers.Dense(128, activation='relu'),  # Adding a fully connected dense layer with 128 units and ReLU activation
        tf.keras.layers.Dense(1, activation='sigmoid')  # Adding the output layer with 1 unit and sigmoid activation for binary classification
        ])
    model.compile(
        loss=tf.keras.losses.BinaryCrossentropy(),  # Using binary crossentropy loss function for binary classification
        optimizer=tf.keras.optimizers.Adam(0.001),  # Using Adam optimizer with learning rate 0.001 less rate as slower convergence of and low rate of parameter update
        metrics=['accuracy']  # Monitoring accuracy metric during training
        )
    model.fit(x_train, y_train, epochs=50)  # Training the model on the training data for 50 epochs
    print(f"Accuracy: {model.evaluate(x_test, y_test)[1] * 100}%")  # Evaluating and printing accuracy of the model
    end=time()
    print('Time to train on GPU',end-start)  # Printing time taken to train the model on GPU

Epoch 1/50
73/73 [==============================] - 8s 70ms/step - loss: 0.6953 - accuracy: 0.5340
Epoch 2/50
73/73 [==============================] - 5s 63ms/step - loss: 0.6696 - accuracy: 0.5869
Epoch 3/50
73/73 [==============================] - 5s 63ms/step - loss: 0.6358 - accuracy: 0.6441
Epoch 4/50
73/73 [==============================] - 5s 63ms/step - loss: 0.6293 - accuracy: 0.6511
Epoch 5/50
73/73 [==============================] - 5s 63ms/step - loss: 0.6080 - accuracy: 0.6775
Epoch 6/50
73/73 [==============================] - 5s 64ms/step - loss: 0.6013 - accuracy: 0.6736
Epoch 7/50
73/73 [==============================] - 5s 64ms/step - loss: 0.5850 - accuracy: 0.6797
Epoch 8/50
73/73 [==============================] - 5s 65ms/step - loss: 0.5654 - accuracy: 0.6931
Epoch 9/50
73/73 [==============================] - 5s 64ms/step - loss: 0.5829 - accuracy: 0.6805
Epoch 10/50
73/73 [==============================] - 5s 63ms/step - loss: 0.5232 - accuracy: 0.7213
Epoch 11/

In [10]:
# save model
model.save("model.keras")  # Saving the trained model